In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## First, I will define the derivatives to integrate.

In [ ]:
def dydx(x,y):
    
    #set the derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs

## Then, I will declare the 5th order RK method.

In [ ]:
def rk5_try(x,y,dydx,h,y_derivs):
    
    ai = array([0, 0.2, 0.3, 0.6, 1.0, 0.875])
    
    ci = array([37.0/378.0, 0.0, 250.0/621.0, 125.0/594.0, 0.0, 512.0/1771.0])
    
    dci = array([ci[0]-28255.0/27648.0, 0.0, ci[2]-18575.0/48384.0, ci[3]-13525.0/55296.0, -277.00/14336.0, ci[5]-0.25])
   
    bs = array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
               [0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
               [3.0/40.0, 9.0/40.0, 0.0, 0.0, 0.0, 0.0],
               [0.3, -0.9, 1.2, 0.0, 0.0, 0.0],
               [-11.0/54.0, 2.5, -70.0/27.0, 35.0/27.0, 0.0, 0.0],
               [1631.0/55296.0, 175.0/512.0, 575.0/13824.0, 44275.0/110592.0, 253.0/4096.0, 0.0])
    
    k0 = h*dydx                                                                              #1st step
    k1 = h*y_derivs(y + bs[1,0]*k0, x + ai[1]*h)                                             #2nd step
    k2 = h*y_derivs(y + bs[2,0]*k0+bs[2,1]*k1, x + ai[2]*h)                                  #3rd step
    k3 = h*y_derivs(y + bs[3,0]*k0+bs[3,1]*k1+bs[3,2]*k2, x + ai[3]*h)                       #4th step
    k4 = h*y_derivs(y + bs[4,0]*k0+bs[4,1]*k1+bs[4,2]*k2+bs[4,3]*k3, x + ai[4]*h)            #5th step
    k5 = h*y_derivs(y + bs[5,0]*k0+bs[5,1]*k1+bs[5,2]*k2+bs[5,3]*k3+bs[5,4]*k4, x + ai[5]*h) #6th step
    
    yout = y + ci[0]*k0+ci[2]*k2+ci[3]*k3+ci[5]*k5
    
    yerr = dci[0]*k0+dci[2]*k2+dci[3]*k3+dci[4]*k4+dci[5]*k5
    
    return (yout, yerr)

## Next, I will define an adaptive step size control driver for RK5.

In [ ]:
def rk5_step(x,y,htry,y_derivs,accuracy):
    
    dydx = y_derivs(y,x) #this calculates the derivatives for the new step
                         #a good way to determine the desired accuracy
    yscal = abs(y[:]) + abs(dydx[:]*htry) + 1e-3
    
    h = htry #set the step size to the initial trial value
    
    while True:
        (y_new, yerr) = rk5_try(x,y,dydx,h,y_derivs) #take a step
        errmax = max(abs(yerr/yscal))/accuracy #this is the max error scaled to required tolerance
        
        if (errmax <= 1.0): break   #step succeeded --> compute size of next step
        h_new = 0.9*h/errmax**0.25  #truncation error too large, reduce step size
        if abs(h_new) < 0.1*abs(h): #if step might get too small
            h_new = 0.1*h           #take at most a 10x smaller step
        h = h_new
        if (x+h == x):
            print ("ERROR: Step size underflow in RKStep.")
            
    if errmax < 2.e-4:  #step is way too small
        h_next = 5.0*h  #increase the step size 5x
    else:
        h_next = 0.9*h/errmax**0.2 #step size is too small
                                   #next time, increase it with the delta^0.2 power
    return (x+h, y_new, h_next)

## Now I will perform the integration.

In [ ]:
if __name__ == '__main__':
    t_start = 0    #first time
    t_stop = 200   #elapsed time in dimensionless units
    h = 0.1        #initial time step
    accuracy = 1e-7
    y = y_derivs([0,1.0])
    
    tc = t_start
    while (tc <= t_stop):
        (tc,y,h) = rk5_step(tc,y,h,accuracy)

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)